In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 5.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1c15dc68a0df6cac6d990cb0ad28051800415e4dacb421e29ec8f3ca8651a5b2
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [3]:
!pip install flair==0.10
!pip install seqeval
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 322 kB 5.1 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.5 MB/s 
     |████████████████████████████████| 48 kB 5.3 MB/s 
     |████████████████████████████████| 4.2 MB 38.5 MB/s 
     |████████████████████████████████| 46 kB 4.0 MB/s 
     |████████████████████████████████| 981 kB 44.0 MB/s 
     |████████████████████████████████| 19.7 MB 1.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 788 kB 49.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 6.6 MB 47.9 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
  Created wheel for gdown: filen

In [4]:
# !unrar x -Y "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [5]:
import pandas as pd
import re
import numpy as np

In [6]:
ROOT = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen"
MODEL = "BiLSTM-CRF-fasttext-sg"
EMBEDDINGS = "wiki.id.bin"
FINAL_MODEL_NAME = "fasttext-sg"

In [7]:
# !unrar x "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [8]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'kata', 1: 'label'}

# this is the folder in which train, test and dev files reside

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(f"{ROOT}/nested", columns,
                              train_file='flair-train.txt',
                              test_file='flair-test.txt',
                              dev_file='flair-test.txt')

2022-05-30 11:53:24,047 Reading data from /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested
2022-05-30 11:53:24,049 Train: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-train.txt
2022-05-30 11:53:24,053 Dev: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt
2022-05-30 11:53:24,057 Test: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt


In [9]:
print(corpus.train[0].to_tagged_string('label'))

Dan <O+O> jika <O+O> orang <O+O> yang <O+O> berhutang <O+O> itu <O+O> dalam <O+O> kesukaran <O+O> maka <O+O> berilah <O+O> tangguh <O+O> sampai <O+O> dia <O+O> berkelapangan <O+O> Dan <O+O> menyedekahkan <O+O> sebagian <O+O> atau <O+O> semua <O+O> utang <O+O> itu <O+O> lebih <O+O> baik <O+O> bagimu <O+O> jika <O+O> kamu <O+O> mengetahui <O+O>


In [10]:
tag_type = 'label'

tag_dictionary = corpus.make_label_dictionary(tag_type)

2022-05-30 11:53:27,345 Computing label dictionary. Progress:


100%|██████████| 763/763 [00:00<00:00, 5506.34it/s]

2022-05-30 11:53:27,494 Corpus contains the labels: label (#29729)
2022-05-30 11:53:27,499 Created (for label 'label') Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [11]:
print(tag_dictionary)

Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [12]:
import gensim
import fasttext
import fasttext.util

In [13]:
from flair.embeddings import FastTextEmbeddings

embedding = FastTextEmbeddings(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/{EMBEDDINGS}')

FastText(vocab=300686, size=300, alpha=0.025)


In [14]:
# from flair.data import Sentence
# embedding.embed(Sentence('12'))

In [15]:
# import fasttext.util
# ft = fasttext.load_model('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
# ft.get_word_vector('12')

In [16]:
import requests, os
session = requests.Session()
session.verify = False
session.trust_env = False
os.environ['CURL_CA_BUNDLE']="" # or whaever other is interfering with 

from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings, FastTextEmbeddings
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): FastTextEmbeddings('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}',
              learning_rate=0.1,
              max_epochs=200,
              )

2022-05-19 07:36:36,078 ----------------------------------------------------------------------------------------------------
2022-05-19 07:36:36,080 Model: "SequenceTagger(
  (embeddings): FastTextEmbeddings('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-05-19 07:36:36,082 ----------------------------------------------------------------------------------------------------
2022-05-19 07:36:36,083 Corpus: "Corpus: 763 train + 763 dev + 763 test sentences"
2022-05-19 07:36:36,085 ----------------------------------------------------------------------------------------------------
2022-05-19 07:36:36,086 Parameters:
2022-05-19 07:36:3

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there is a model from previous trainings"


2022-05-19 07:36:36,836 ----------------------------------------------------------------------------------------------------
2022-05-19 07:36:37,329 epoch 1 - iter 2/24 - loss 1.73603693 - samples/sec: 130.39 - lr: 0.100000
2022-05-19 07:36:37,621 epoch 1 - iter 4/24 - loss 1.18458089 - samples/sec: 221.47 - lr: 0.100000
2022-05-19 07:36:37,994 epoch 1 - iter 6/24 - loss 0.90449499 - samples/sec: 172.45 - lr: 0.100000
2022-05-19 07:36:38,269 epoch 1 - iter 8/24 - loss 0.78141984 - samples/sec: 234.87 - lr: 0.100000
2022-05-19 07:36:38,706 epoch 1 - iter 10/24 - loss 0.68339129 - samples/sec: 147.27 - lr: 0.100000
2022-05-19 07:36:39,029 epoch 1 - iter 12/24 - loss 0.62925969 - samples/sec: 199.52 - lr: 0.100000
2022-05-19 07:36:39,316 epoch 1 - iter 14/24 - loss 0.59572733 - samples/sec: 224.45 - lr: 0.100000
2022-05-19 07:36:39,687 epoch 1 - iter 16/24 - loss 0.56299459 - samples/sec: 173.60 - lr: 0.100000
2022-05-19 07:36:40,015 epoch 1 - iter 18/24 - loss 0.53414504 - samples/sec: 1

{'dev_loss_history': [tensor(0.2624, device='cuda:0'),
  tensor(0.1522, device='cuda:0'),
  tensor(0.1178, device='cuda:0'),
  tensor(0.1055, device='cuda:0'),
  tensor(0.0897, device='cuda:0'),
  tensor(0.0862, device='cuda:0'),
  tensor(0.0939, device='cuda:0'),
  tensor(0.0681, device='cuda:0'),
  tensor(0.0633, device='cuda:0'),
  tensor(0.0658, device='cuda:0'),
  tensor(0.0614, device='cuda:0'),
  tensor(0.0631, device='cuda:0'),
  tensor(0.0559, device='cuda:0'),
  tensor(0.0545, device='cuda:0'),
  tensor(0.0528, device='cuda:0'),
  tensor(0.0580, device='cuda:0'),
  tensor(0.0501, device='cuda:0'),
  tensor(0.0477, device='cuda:0'),
  tensor(0.0491, device='cuda:0'),
  tensor(0.0433, device='cuda:0'),
  tensor(0.0457, device='cuda:0'),
  tensor(0.0509, device='cuda:0'),
  tensor(0.0489, device='cuda:0'),
  tensor(0.0436, device='cuda:0'),
  tensor(0.0403, device='cuda:0'),
  tensor(0.0417, device='cuda:0'),
  tensor(0.0410, device='cuda:0'),
  tensor(0.0439, device='cuda:0'),


In [17]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}/best-model.pt')

2022-05-30 11:54:33,622 loading file /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-fasttext-sg/best-model.pt


In [18]:
FINAL_MODEL_NAME

'fasttext-sg'

In [19]:
# create example sentence
sentence = Sentence('yaitu mereka orang-orang yang dimurkai allah')
# predict the tags
model.predict(sentence)

print(sentence.to_tagged_string())

yaitu <O+O> mereka <O+O> orang-orang <B-PER+B-PER> yang <I-PER+I-PER> dimurkai <I-PER+I-PER> allah <I-PER+I-PER>


In [20]:
for entity in sentence.get_spans('label'):
    print(entity)

Span [1]: "yaitu"   [− Labels: O+O (0.989)]
Span [2]: "mereka"   [− Labels: O+O (0.8908)]
Span [3,4,5,6]: "orang-orang yang dimurkai allah"   [− Labels: PER+I-PER (0.9463)]


In [21]:
sentence.to_dict('label')

{'entities': [{'end_pos': 5,
   'labels': [O+O (0.989)],
   'start_pos': 0,
   'text': 'yaitu'},
  {'end_pos': 12, 'labels': [O+O (0.8908)], 'start_pos': 6, 'text': 'mereka'},
  {'end_pos': 44,
   'labels': [PER+I-PER (0.9463)],
   'start_pos': 13,
   'text': 'orang-orang yang dimurkai allah'}],
 'labels': [],
 'text': 'yaitu mereka orang-orang yang dimurkai allah'}

In [22]:
# from ast import literal_eval

# test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

# test_data['sent'] = test_data['sent'].apply(literal_eval)
# test_data['tag'] = test_data['tag'].apply(literal_eval)
# test_data['pos'] = test_data['pos'].apply(literal_eval)

# ii_lab = []
# for i, d in test_data.iterrows() :
#   sent = ' '.join(d.sent)

#   sentence = Sentence(sent)

#   model.predict(sentence)

#   entities = sentence.get_labels('label')
#   lab_idx = []
#   entity_dict = {}
#   for entity in entities:
#       entity_in_str = str(entity)
#       idx = re.findall(r"\d+", entity.identifier)
#       idx = [int(i) for i in idx]
#       idx = [i - 1 for i in idx]
#       if "+I" in entity_in_str : 
#         entity_dict[idx[0]] = entity_in_str.split()[0]

#       lab_idx = lab_idx + idx

#   new_lab_idx = []
#   for i in range(len(d.sent)) : 
#     if i not in lab_idx : 
#       new_lab_idx.append(-999)

#     else : 
#       new_lab_idx.append(i)

#   final_lab_idx = []
#   i = 0
#   for idx in new_lab_idx :
#     if idx == -999 : 
#       final_lab_idx.append('O')
#       i = 0
#     elif idx in entity_dict : 
#       final_lab_idx.append(entity_dict[idx])
#     else : 
#       i += 1
#       if i == 1 : 
#         final_lab_idx.append('B-PER')
#       else : 
#         final_lab_idx.append('I-PER')

#   ii_lab.append(final_lab_idx)


In [23]:
from ast import literal_eval

test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)

test_data = test_data.sort_values(['sentence #'])

ii_lab = []
for i, d in test_data.iterrows() :
  sent = ' '.join(d.sent)

  sentence = Sentence(sent)

  model.predict(sentence)

  t = sentence.to_tagged_string()

  word = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][::2] 
  tag = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][1::2] 
  
  ii_lab.append(tag)


In [24]:
# t = sentence.to_tagged_string()
# import re

# word = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][::2] 
# tag = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][1::2] 
# len(word)

In [25]:
test_data['pred'] = ii_lab
y_pred = ii_lab
y_true = test_data['tag'].to_list()

In [26]:
def get_level_token(seqs, max_lev=2, level=0) : 
  new_all_level = []
  for seq in seqs : 
    new_all_level_i = []
    for tag in seq : 
      new_tag = []
      tag_per_level = tag.split('+')
      for lev in range(max_lev) :
        if lev + 1 > len(tag_per_level) : 
          new_tag.append(tag_per_level[-1])
        else : 
          new_tag.append(tag_per_level[lev])
      new_all_level_i.append(new_tag)
    new_all_level.append(new_all_level_i)
  
  new_seq = []
  for seq in new_all_level :
    seq_i = [] 
    for tag in seq : 
      seq_i.append(tag[level])
    new_seq.append(seq_i)
  
  return new_seq

y_true_0 = get_level_token(y_true, max_lev=2, level=0)
y_pred_0 = get_level_token(y_pred, max_lev=2, level=0)

In [27]:
from seqeval.metrics import classification_report
print(classification_report(y_true_0, y_pred_0))

              precision    recall  f1-score   support

         PER       0.83      0.79      0.81       222

   micro avg       0.83      0.79      0.81       222
   macro avg       0.83      0.79      0.81       222
weighted avg       0.83      0.79      0.81       222



In [28]:
y_true_1 = get_level_token(y_true, max_lev=2, level=1)
y_pred_1 = get_level_token(y_pred, max_lev=2, level=1)

from seqeval.metrics import classification_report
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

         PER       0.78      0.74      0.76       222

   micro avg       0.78      0.74      0.76       222
   macro avg       0.78      0.74      0.76       222
weighted avg       0.78      0.74      0.76       222



In [29]:
y_test_token = []
for i in y_true : 
  y_test_token = y_test_token + i

y_pred_token = []
for i in y_pred : 
  y_pred_token = y_pred_token + i

from sklearn.metrics import classification_report
print(classification_report(y_test_token, y_pred_token))


              precision    recall  f1-score   support

 B-PER+B-PER       0.95      0.90      0.92       222
 I-PER+I-PER       0.94      0.82      0.87       444
     O+I-PER       0.80      0.32      0.46       114
         O+O       0.97      1.00      0.98      6243

    accuracy                           0.97      7023
   macro avg       0.92      0.76      0.81      7023
weighted avg       0.97      0.97      0.97      7023



In [30]:
test_data = test_data.explode(['sent', 'tag', 'pred'])
test_data = test_data[['juz', 'no_ayat', 'sent', 'tag', 'pred']]
test_data

,juz,no_ayat,sent,tag,pred
68,1,2,Segala,O+O,O+O
68,1,2,puji,O+O,O+O
68,1,2,bagi,O+O,O+O
68,1,2,Allah,O+O,O+O
68,1,2,Tuhan,O+O,O+O
...,...,...,...,...,...
124,6,155,bertakwalah,O+O,O+O
124,6,155,agar,O+O,O+O
124,6,155,kamu,O+O,O+O
124,6,155,diberi,O+O,O+O


In [31]:
test_data.to_csv(f"{ROOT}/nested/results-prediction/bilstm.csv", index=False)